In [1]:
import ee

# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

import geemap
import json
from ipyleaflet import GeoJSON, Marker, MarkerCluster
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from skimage import exposure
import cv2
import os
import datetime
from dl_landsat import LANDSAT_8, apply_scale_factors_8, mask_clouds, export_to_drive, export_to_numpy

In [2]:
CITY_COORDS = [36.86905523581975, -1.2815372605877613]
CITY = ee.Geometry.Point(CITY_COORDS) # Coord format: E, N 

YEAR = 2021

Map = geemap.Map()

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
RELEVANT_MONTHS_WINTER = [1, 2, 3, 4, 10, 11, 12]
RELEVANT_MONTHS_SUMMER = [5, 6, 7, 8, 9]
RELEVANT_MONTHS = RELEVANT_MONTHS_WINTER

ls_read = None
for month in RELEVANT_MONTHS:
    if ls_read is None:
        ls_read = (ee.ImageCollection(LANDSAT_8)
                   .map(apply_scale_factors_8)
                   .filterBounds(CITY)
                   .filter(ee.Filter.calendarRange(YEAR, YEAR+1, "year"))
                   .filter(ee.Filter.calendarRange(month, month+1, "month"))
                   .map(mask_clouds)
                   )
    else:
        ls_read_ = (ee.ImageCollection(LANDSAT_8)
                   .map(apply_scale_factors_8)
                   .filterBounds(CITY)
                   .filter(ee.Filter.calendarRange(YEAR, YEAR+1, "year"))
                   .filter(ee.Filter.calendarRange(month, month+1, "month"))
                   .map(mask_clouds)
                   )
        ls_read = ls_read.merge(ls_read_)

ls_median = ls_read.median()

data_available = ls_median.getInfo() is not None

print("Data Available: ", data_available)
if data_available:
    print("Number of exported Months:")
    print(ls_read.size().getInfo())
    time_stamps = ls_read.aggregate_array("system:time_start").getInfo()
    dates = list(
        map(lambda x: datetime.datetime.fromtimestamp(x / 1e3).strftime("%Y/%m/%d %H:%M"), time_stamps))
    print("Images Exported:")
    for date in dates:
        print(date)
    vis_params_wc = {"bands": ['Map']}
    vis_params_rgb = {'bands': ['B3', "B2", "B1"], "min": 0, "max": 0.3, 'opacity': 1.0}
    Map.centerObject(CITY, 14)
    Map.addLayer(ls_median, vis_params_rgb, f"Satelite Image")
else:
    print("Error Couldn't find datapoint")

Data Available:  True
Number of exported Months:
27
Images Exported:
2014/01/24 11:00
2014/02/09 11:00
2014/02/25 10:59
2014/02/09 11:00
2014/02/25 10:59
2013/04/11 11:03
2014/04/14 10:59
2013/04/11 11:03
2013/05/13 11:01
2014/04/14 10:59
2014/05/16 10:58
2013/10/04 11:00
2013/10/20 11:00
2013/11/05 11:00
2013/11/21 11:00
2014/10/07 10:59
2014/10/23 10:59
2013/11/05 11:00
2013/11/21 11:00
2013/12/07 11:00
2013/12/23 11:00
2014/12/10 10:59
2014/12/26 10:59
2013/12/07 11:00
2013/12/23 11:00
2014/12/10 10:59
2014/12/26 10:59


In [6]:
YEARS = list(range(2013, 2022))
base_folder_name_1 = "Pohang_Si_Winter"
base_folder_name_2 = "Pohang_Si_Summer"

export_to_drive(CITY_COORDS, years=YEARS, months=RELEVANT_MONTHS_WINTER, base_folder_name=base_folder_name_1)
export_to_drive(CITY_COORDS, years=YEARS, months=RELEVANT_MONTHS_SUMMER, base_folder_name=base_folder_name_2)

In [10]:
band_list = [1, 2, 3, 4, 5, 6, 7, 8]
export_to_numpy(years=YEARS, base_folder_name=base_folder_name_1, band_list=band_list)
export_to_numpy(years=YEARS, base_folder_name=base_folder_name_2, band_list=band_list)

(8, 8, 665, 665)
(1, 8, 665, 665)


In [4]:
from dl_landcover import main

coords = [36.86905523581975, -1.2815372605877613]
main(coords, 'data/nairobi_images_36-86_1-28', 'data/nairobi_images_36-86_1-28/data_raw.npy')

(9, 8, 665, 665)


KeyboardInterrupt: 